In [1]:
!pip install ckanapi
from ckanapi import *
import csv
import requests

api = RemoteCKAN('https://dadosabertos.capes.gov.br')
dataStore = api.action.package_list()

In [2]:
#Função para filtrar a lista de datasets, colocando os que se adequam ao filtro
def filterCatch(data, filter):
  return [str for str in data if
          any(sub in str for sub in filter)]

#Função para filtrar a lista de datasets, retirando os que se adequam ao filtro
def filterMiss(data, filter):
  return [str for str in data if
          not any(sub in str for sub in filter)]

In [3]:
import logging
import time
import os
from minio import Minio

client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )

bucket_name="raw"
found = client.bucket_exists(bucket_name)
if not found:
    client.make_bucket(bucket_name)

In [4]:
#FUNÇÃO QUE FAZ O DOWNLOAD E JOGA AUTOMATICAMENTE OS ARQUIVOS NO MinIO
def download_dataset_to_Minio(datastore, datasets_list, client_Minio, max_attempts=3):
    # Lista todos os objetos no bucket
    objects = client_Minio.list_objects('raw', prefix='')
    # Remove cada objeto do Minio
    for obj in objects:
        client_Minio.remove_object('raw', obj.object_name)

    #PERCORRENDO PELA DATASTORE DO CAPES E PELA LISTA DOS DATASETS DE INTERESSE
    for database, datasets in zip(datastore, datasets_list):
        for csv in datasets:
            #PEGANDO O ARQUIVO DA API E CRIANDO OS PARAMETROS QUE SERAO UTILIZADOS NO DOWNLOAD E SALVAMENTO NO MinIO
            archive = api.action.package_show(id=database)
            link_archive = archive['resources'][csv]['url']
            name_archive = f"{database}/{archive['resources'][csv]['description']}.csv"
            try:
                for attempt in range(max_attempts):
                    try:
                        print(f"Tentativa {attempt + 1}/{max_attempts}")
                        !wget -O {'tmp.csv'} --retry-connrefused {link_archive} #DOWNLOAD COMO ARQUIVO TEMPORARIO
                        client_Minio.fput_object('raw', "CAPES/" + name_archive, 'tmp.csv') #UPLOAD DO ARQUIVO
                        os.remove('tmp.csv') #REMOÇÃO DO ARQUIVO TEMPORARIO
                        break  # Download bem-sucedido
                        
                    except ConnectionError as e:
                        logging.error(f"Erro de conexão ao baixar dataset {dataset} (tentativa {attempt + 1}/{max_attempts}): {e}")
                        time.sleep(60)  # Aguardar 1 minuto antes de tentar novamente
            
                                # Tentar continuar com -c em caso de erro de conexão
                        !wget -c {name_archive} {link_archive}
            
                    except Exception as e:
                        logging.error(f"Erro inesperado ao baixar dataset {dataset}: {e}")
            
            except Exception as e:
                logging.error(f"Erro ao obter informações do dataset {dataset}: {e}")

In [5]:
filters=["2021-a-2024"]
database=filterCatch(dataStore,filters)
print(*database,sep="\n")

2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil
2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-detalhes-da-producao-intelectual-artistica-de-programas-de-pos-graduacao-stricto-sensu
2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao
2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu
2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil
2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu

In [6]:
filters=["artistica"]
database=filterMiss(database,filters)
print(*database,sep="\n")

2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil
2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao
2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu
2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil
2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil


In [7]:
#CRIANDO MANUALMENTE A LISTA DE DATASETS DE INTERESSE
datasets_list = [[8,10,12,14,17,20,22,34,36,40,48,50,52],
         [0,2],
         [0,2],
         [0,2,4,6],
         [2,8,10,14,22,24,26],
         [0,2],
         [0,2],
         [0,2],
         [0,2],
         [8,10,12,14,22,28,30,34,42,44,46],
         [0,2],
         [0,2]]

In [8]:
download_dataset_to_Minio(database, datasets_list, client)

Tentativa 1/3
--2024-09-22 10:32:12--  https://dadosabertos.capes.gov.br/dataset/d7ee8c2b-253b-4619-884c-18f376b619ac/resource/ab7db4a8-3d9c-4628-bc66-1ab0ac64f1ac/download/br-capes-colsucup-prod-autor-2021a2024-2023-10-31-bibliografica-anais-2021.csv
Resolving dadosabertos.capes.gov.br (dadosabertos.capes.gov.br)... 200.130.18.195, 200.130.18.196, 2001:12f0:b27:1004::246, ...
Connecting to dadosabertos.capes.gov.br (dadosabertos.capes.gov.br)|200.130.18.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103264478 (98M) [text/csv]
Saving to: ‘tmp.csv’

tmp.csv             100%[===================>]  98.48M  4.79MB/s    in 21s     

2024-09-22 10:32:34 (4.62 MB/s) - ‘tmp.csv’ saved [103264478/103264478]

Tentativa 1/3
--2024-09-22 10:32:35--  https://dadosabertos.capes.gov.br/dataset/d7ee8c2b-253b-4619-884c-18f376b619ac/resource/f2e18646-c4c0-45f8-b605-cf95d64ed9d4/download/br-capes-colsucup-prod-autor-2021a2024-2023-10-31-bibliografica-anais-2022.csv
Resolvin

In [9]:
#CODIGO PARA A LISTAGEM DOS DATASETS DISPONIVEIS EM CSV DAS CATEGORIAS DOS DADOS ABERTOS DO CAPES
for num in range(0, len(database)):#PARA CADA CATEGORIA
    archive = api.action.package_show(id=database[num])
    print(database[num])#SE MOSTRA QUAL ESTA NO MOMENTO
    for i in range(0, len(archive['resources'])):#E PARA CADA RECURSO, DIZ-SE POR CADA ARQUIVO
        if(archive['resources'][i]['format'] == "CSV"):#SE O TIPO FOR CSV
            print(f"{i} {archive['resources'][i]['description']}")#ENTAO MOSTRA A DESCRIÇÃO
    print('\n')
            

2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil
0 Ano Base - 2021 a 2022, Artística Subtipo Artes Cênicas
2 Ano Base - 2021 a 2022, Artística Subtipo Artes Visuais
4 Ano Base - 2021 a 2022, Artística Subtipo Música
6 Ano Base - 2021 a 2022, Artística Subtipo Outros
8 Ano Base - 2021, Bibliográfica Subtipo Trabalho em Anais
10 Ano Base - 2022, Bibliográfica Subtipo Trabalho em Anais
12 Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista
14 Ano Base - 2021, Bibliográfica Subtipo Artigo em Periódico
17 Ano Base - 2022, Bibliográfica Subtipo Artigo em Periódico
20 Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro
22 Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro
24 Ano Base - 2021 a 2022, Bibliográfica Subtipo Partitura Musical
26 Ano Base - 2021 a 2022, Bibliográfica Subtipo Tradução
28 Ano Base - 2021 a 2022, Técnica Subtipo Apresentação de Trabalho
30 Ano Base - 2021 a 2022, Técnica Subtipo Cartas, Mapas ou Simi